In [ ]:
# Import necessary libraries
import cv2
import numpy as np

# Define path to the input video file and YOLOv8 configuration and weights
video_path = '/data/input_video.mp4'  # Example path, adjust according to actual file location
output_video_path = '/data/processed_video.mp4'
yolov8_weights = 'yolov8.weights'  # Adjust with the actual path to YOLOv8 weights
yolov8_config = 'yolov8.cfg'  # Adjust with the actual path to YOLOv8 config file
classes_file = 'coco.names'  # COCO class names file

# Load the COCO class names
with open(classes_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Load YOLOv8 network
net = cv2.dnn.readNet(yolov8_weights, yolov8_config)
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Initialize video
video = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(video.get(3))
frame_height = int(video.get(4))
fps = int(video.get(cv2.CAP_PROP_FPS))

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while True:
    ret, frame = video.read()
    if not ret:
        break

    height, width, channels = frame.shape

    # Convert frame to blob
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    # Process detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "car":
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-maximum Suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes on the frame
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # Write the frame into the file
    out.write(frame)

# Release everything if job is finished
video.release()
out.release()
cv2.destroyAllWindows()

# The paths to the yolov8.weights, yolov8.cfg, and coco.names files need to be adjusted to where you have stored these files.
# This script assumes those files are correctly set up and accessible.
